# DATA EXPLORATION GAS SALES

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List

In [ ]:
# Define folder path
DATA_FOLDER_PATH = f"{os.path.dirname(os.getcwd())}/data/raw/"
GAS_DATA_FOLDER_PATH = f"{DATA_FOLDER_PATH}/gas/"

In [ ]:
# Sample file path
DATA_FILE_PATH_BY_YEAR = os.path.join(GAS_DATA_FOLDER_PATH, '2019')
FILES = os.listdir(DATA_FILE_PATH_BY_YEAR)
DEMO_DATA_FILE_PATH = os.path.join(DATA_FILE_PATH_BY_YEAR, FILES[0])
print(DEMO_DATA_FILE_PATH)


In [ ]:
# Read df
sample_df = pd.read_csv(DEMO_DATA_FILE_PATH, low_memory=False, thousands=',')
sample_df.head(10)

In [ ]:
# Show info
print(sample_df.shape)
sample_df.info()

In [ ]:
# Remove totals
def remove_totals(data_df: pd.DataFrame, column: str) -> pd.DataFrame:
    filtered_df = data_df[data_df[column].notna()]
    return filtered_df

sample_df = remove_totals(sample_df,"Filial")
sample_df.shape

In [ ]:
# Convert type
sample_df.convert_dtypes()

In [ ]:
# Get date
def keydate_to_date(data_df: pd.DataFrame, column: str = "%Key_FechaID") -> pd.Series:
    date_series = pd.to_datetime(data_df[column].astype(str), format = "%Y%m%d")
    return date_series

sample_df["Date"] = keydate_to_date(sample_df, "%Key_FechaID")

In [ ]:
# Remove columns
COLUMNS_TO_REMOVE = ["Empresa", "%Key_Ticket","%Key_Folio", "%Key_Cliente", "%Key_FechaID"]
sample_df.drop(columns = COLUMNS_TO_REMOVE, inplace=True)
sample_df.sample(5)

In [ ]:
file_filters = os.path.join(DATA_FOLDER_PATH, 'filiales_estados.csv')
filters_df = pd.read_csv(file_filters, low_memory=False, thousands=',')
filters_df.dropna(axis=1, how="all", inplace=True)
sample_df = pd.merge(filters_df, sample_df, on="Filial")
sample_df.sample(5)

In [ ]:
# Filter by state (NUEVO LEON, TAMAULIPAS)
def filter_by_state(data_df: pd.DataFrame, states: List[str] = ["NUEVO LEON", "TAMAULIPAS"]):
    filtered_df = data_df[data_df["Estado"].isin(states)]
    filtered_df.reset_index(drop=True)
    return filtered_df

sample_df = filter_by_state(sample_df)
print(sample_df.shape)
sample_df.sample(5)


In [ ]:
def get_sale_type(sale_point: str) -> str:
    if "ATQ" in sale_point:
        sale_type = "ATQ"
    else:
        sale_type = "EC"
    return sale_type

sample_df["Tipo"] = sample_df["Punto de venta"].apply(lambda x: get_sale_type(x))
sample_df.head(5)

In [ ]:
def process_year_data(input_data_path: str, year: str, output_path: str = "output/") -> pd.DataFrame:
    # Init year_df
    df_year = pd.DataFrame([])
    # Define paths
    year_data_path = f"{input_data_path}/{year}"
    output_path = f"{output_path}/{year}"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Read filial-state data
    file_filters = os.path.join(DATA_FOLDER_PATH, 'filiales_estados.csv')
    filters_df = pd.read_csv(file_filters, low_memory=False, thousands=',')
    filters_df.dropna(axis=1, how="all", inplace=True)
    # Get gas year data
    year_files = os.listdir(year_data_path)
    # Read each file in year

    for file in year_files:
        # Read month file
        file_path = os.path.join(year_data_path, file)
        m_df = pd.read_csv(file_path, low_memory=False, thousands=',')
        # Remove totals
        m_df = remove_totals(m_df, "Filial")
        # Fix dtypes
        m_df.convert_dtypes()
        # Set state
        m_df = pd.merge(filters_df, m_df, on="Filial")
        # Filter data by state
        m_df = filter_by_state(m_df)
        # Convert date
        m_df["Date"] = keydate_to_date(m_df, "%Key_FechaID")
        # Get sale type
        m_df["Tipo"] = m_df["Punto de venta"].apply(lambda x: get_sale_type(x))
        # Remove unnecesary columns
        m_df.drop(columns = COLUMNS_TO_REMOVE, inplace=True)
        # Export data
        # m_df.to_csv(f'{output_path}/{file}')
        # Append data
        # df_year = df_year.append(m_df)
        df_year = pd.concat([df_year, m_df], ignore_index=True)
    # df_year.to_csv(f'{output_path}/{year}.csv')
    df_year.sort_values(['Date'], ascending=[True], inplace=True)
    return df_year
 

year_sample = process_year_data(GAS_DATA_FOLDER_PATH, "2022")
print(year_sample.shape)
year_sample.head(5)

In [ ]:
YEARS: List[int] = [2017, 2018, 2019, 2020, 2021, 2022]
def get_all_years(input_data_path: str, years: List[str], output_path: str = "output/") -> pd.DataFrame:
    # Init data_df
    full_data_df = pd.DataFrame([])
    for year in years:
        year_df = process_year_data(input_data_path, year, output_path)
        full_data_df = pd.concat([full_data_df, year_df], ignore_index=True)
    # df_year.to_csv(f'{output_path}/{year}.csv')
    full_data_df.sort_values(['Date'], ascending=[True], inplace=True)
    return full_data_df


full_gas_data = get_all_years(GAS_DATA_FOLDER_PATH, YEARS)
print(full_gas_data.shape)
full_gas_data.head(5)

In [ ]:
atq_df = full_gas_data[full_gas_data["Punto de venta"].str.startswith("ATQ")]
print(atq_df["Tipo"].unique())
ec_df = full_gas_data[full_gas_data["Punto de venta"].str.startswith("EC")]
print(ec_df["Tipo"].unique())
other_points_df = full_gas_data[~full_gas_data["Punto de venta"].str.startswith(("EC","ATQ"))]
print(other_points_df["Tipo"].unique())


In [ ]:
temp_data = pd.read_csv(f"{DATA_FOLDER_PATH}/temperature.csv")
temp_data.dropna(axis=1, how="all", inplace=True)
temp_data

In [ ]:
# Add month and year columns
full_gas_data["Mes"] = full_gas_data["Date"].dt.month
full_gas_data["Año"] = full_gas_data["Date"].dt.year


In [ ]:
gas_temp_df = pd.merge(full_gas_data, temp_data, how="left")
gas_temp_df.shape

In [ ]:
occupation_df = pd.read_csv(f"{DATA_FOLDER_PATH}/ocupacion.csv")
occupation_df[["Año", "Mes"]] = occupation_df["Periodo"].str.split("/", expand=True)
occupation_df["Mes"] = occupation_df["Mes"].astype(int)
occupation_df["Año"] = occupation_df["Año"].astype(int)
occupation_df.drop(columns = "Periodo", inplace=True)
occupation_df

In [ ]:
full_df = pd.merge(gas_temp_df, occupation_df, how="left")
full_df.fillna(method="ffill", inplace=True)
full_df.drop(columns=["Mes", "Año"], inplace=True)
full_df.tail(5)


In [ ]:
full_df.columns

## Data Reduction

In [ ]:
full_df.ap.isna().any()

In [ ]:
full_df.shape

In [ ]:
reduced_table = pd.pivot_table(full_df, index=['Filial', 'Estado', 'Region', 'Punto de venta', 'Date',
     'Tipo'], values=['Litros', '$', 'Temp. Promedio', 'Temp. Maxima', 'servicio de alojamiento'],
     aggfunc={'Litros': sum,
          '$': sum,
          'Temp. Promedio': np.mean,
          'Temp. Maxima': np.mean,
          'servicio de alojamiento': np.mean})
reduced_table.reset_index(inplace=True)
reduced_table.sample(5)
                   

In [ ]:
reduced_table.shape

In [ ]:
reduced_table.to_csv(f"output/full_gas_data.csv", index=False)